In [64]:
import sqlite3
import binascii

# SQLite 创建自定义函数
- 创建基本函数
    - conn.create_function(name, params_num, fun_name)
- 创建聚合函数
    - conn.create_aggregate(name, params_num, class_name)

In [65]:
db_name = "./db/exemple_1.db"

In [66]:
def encrypt(mydata):
    crc = str(binascii.crc32(mydata.encode()))
    while len(crc) < 10:
        crc = "0" + crc
    return mydata + crc

In [67]:
def check(mydata):
    if len(mydata) < 11:
        return None
    crc_res = str(binascii.crc32(mydata[:-10].encode()))
    while len(crc_res) < 10:
        crc_res = "0" + crc_res
    if crc_res == mydata[-10:]:
        return mydata[:-10]

In [68]:
# 创建连接
conn = sqlite3.connect(db_name)
# 创建函数
conn.create_function("checkk", 1, check)
# 创建游标
cursor = conn.cursor()

In [69]:
# 创建表
sql_script = """
drop table if exists users;
create table users(id integer, name text);
insert into users values(1, "%s");
insert into users values(2, "%s");
"""

In [70]:
names = ['Alice', 'Bob']
names = tuple(encrypt(name) for name in names) # 加密
sql_script = sql_script % names

In [71]:
# 执行脚本
cursor.executescript(sql_script)

In [72]:
# 查询
cursor.execute("select id, checkk(name) from users")
for row in cursor.fetchall():
    print(row)

(1, 'Alice')
(2, 'Bob')


In [73]:
# 修改下名字，然后再次查询
cursor.execute("update users set name = ? where id = ?", ('dsfsfsfsf34242424', 2))
cursor.execute("select id, checkk(name) from users")
for row in cursor.fetchall():
    print(row)

(1, 'Alice')
(2, None)


In [74]:
conn.commit()
cursor.close()
conn.close()